In [24]:
import os
import numpy as np
import cv2
from keras_facenet import FaceNet
from sklearn.linear_model import LogisticRegression
import joblib

# ----------------------------
# 1. Prepare dataset
# ----------------------------
persons = {
    "Abhinav": ["/content/1000162666.jpg", "/content/abhtest.jpg"],
    "Srikar": ["/content/1000162667.jpg", "/content/Pi7_Tool_Srikar_passport_photo.jpg"],
    "Sukesh": ["/content/1000162665.jpg", "/content/suktest.jpg"]
}

# ----------------------------
# 2. Load FaceNet
# ----------------------------
embedder = FaceNet()

def load_and_preprocess(img_path):
    """Load image and preprocess for FaceNet"""
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found or unreadable: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_embedding(img_path):
    """Get 512-dim FaceNet embedding"""
    img = load_and_preprocess(img_path)
    embedding = embedder.embeddings([img])[0]
    return embedding

# ----------------------------
# 3. Build training data
# ----------------------------
X, y = [], []
labels = list(persons.keys())

for label in labels:
    for img_path in persons[label]:
        try:
            emb = get_embedding(img_path)
            X.append(emb)
            y.append(label)
        except Exception as e:
            print(f"⚠️ Skipping {img_path}: {e}")

X = np.array(X)
y = np.array(y)

print("✅ Data prepared:", X.shape, y.shape)

# ----------------------------
# 4. Train One-vs-All classifiers
# ----------------------------
models = {}
for label in labels:
    binary_y = (y == label).astype(int)  # 1 for this person, 0 for others
    clf = LogisticRegression(max_iter=1000, class_weight="balanced")
    clf.fit(X, binary_y)
    models[label] = clf
    joblib.dump(clf, f"{label}_model.pkl")
    print(f"💾 Saved model for {label}")

# ----------------------------
# 5. Prediction Function
# ----------------------------
def predict(img_path, models, threshold=0.3):
    try:
        emb = get_embedding(img_path).reshape(1, -1)
    except Exception as e:
        return "Error", str(e)

    scores = {label: model.predict_proba(emb)[0][1] for label, model in models.items()}
    best_label = max(scores, key=scores.get)
    best_score = scores[best_label]

    if best_score >= threshold:
        return best_label, scores
    else:
        return "Unknown", scores

# ----------------------------
# 6. Test predictions
# ----------------------------
for person, imgs in persons.items():
    for test_img in imgs:
        result = predict(test_img, models)
        print(f"🧪 {person} test: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
✅ Data prepared: (6, 512) (6,)
💾 Saved model for Abhinav
💾 Saved model for Srikar
💾 Saved model for Sukesh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
🧪 Abhinav test: ('Abhinav', {'Abhinav': np.float64(0.591450086448346), 'Srikar': np.float64(0.3980138640632825), 'Sukesh': np.float64(0.43071583658500584)})
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🧪 Abhinav test: ('Abhinav', {'Abhinav': np.float64(0.5887685733212001), 'Srikar': np.float64(0.4097548538634941), 'Sukesh': np.float64(0.42158407965428396)})
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
🧪 Srikar test: ('Srikar', {'Abhinav': np.float64(0.4451725896570567), 'Srikar': np.float64(0.5659397788678654), 'Sukesh': np.float64(0.41042501896482975)})
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🧪 Srikar test: ('Srikar', {'Abhinav': np.flo

In [25]:
import os
import numpy as np
import cv2
from keras_facenet import FaceNet
from sklearn.linear_model import LogisticRegression
import joblib

# Training data
persons = {
    "Abhinav": ["/content/1000162666.jpg", "/content/abhtest.jpg"],
    "Srikar": ["/content/1000162667.jpg", "/content/Pi7_Tool_Srikar_passport_photo.jpg"],
    "Sukesh": ["/content/1000162665.jpg", "/content/suktest.jpg"]
}

# Load FaceNet
embedder = FaceNet()

def load_and_preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_embedding(img_path):
    img = load_and_preprocess(img_path)
    return embedder.embeddings([img])[0]

# Prepare embeddings
X, y = [], []
labels = list(persons.keys())
for label in labels:
    for img_path in persons[label]:
        emb = get_embedding(img_path)
        X.append(emb)
        y.append(label)

X, y = np.array(X), np.array(y)

# Train one-vs-all logistic regression
models = {}
for label in labels:
    binary_y = (y == label).astype(int)
    clf = LogisticRegression(max_iter=1000, class_weight="balanced")
    clf.fit(X, binary_y)
    models[label] = clf
    joblib.dump(clf, f"{label}_model.pkl")

# Save label list
joblib.dump(labels, "labels.pkl")
print("✅ Models and labels saved!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
✅ Models and labels saved!


In [27]:
import cv2
import numpy as np
import joblib
from keras_facenet import FaceNet

# Load models
labels = joblib.load("labels.pkl")
models = {label: joblib.load(f"{label}_model.pkl") for label in labels}

# FaceNet embedder
embedder = FaceNet()

def load_and_preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_embedding(img_path):
    img = load_and_preprocess(img_path)
    return embedder.embeddings([img])[0]

def predict(img_path, models, threshold=0.3):
    emb = get_embedding(img_path).reshape(1, -1)
    scores = {label: model.predict_proba(emb)[0][1] for label, model in models.items()}
    best_label = max(scores, key=scores.get)
    best_score = scores[best_label]
    if best_score >= threshold:
        return best_label, scores
    else:
        return "Unknown", scores

# Test
test_img = "/content/Pi7_Tool_Srikar_passport_photo.jpg"  # replace with your path
label, scores = predict(test_img, models)
print("Prediction:", label)
print("Scores:", scores)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Prediction: Srikar
Scores: {'Abhinav': np.float64(0.35529653933474575), 'Srikar': np.float64(0.5902395217354187), 'Sukesh': np.float64(0.47469165588357526)}
